# **Installing packages and importing libraries**

In [17]:

!pip install -q pyomo
from pyomo.environ import *
import numpy as np

!apt-get install -y -qq glpk-utils

!apt-get install -y -qq coinor-cbc



# Exercise 1 Problem Statement with Model and both solver

So our optimization problem (OP) reduces to solving the following problem.

In the below problem c^{T} is our coefficient vector and x is our variable vector. 

A is the matrix of coefficients of constraints

b is the vector of rhs of the constraints

l is the vector of lower bounds of x



 \begin{array}{l}
Min\ \ \ \ \ \ \ \ c^{T} x\\
\\
s.t\ \ \ \ \ \ \ \ \ Ax\ \leqslant b\\
\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ x\ \geqslant \ l\ \ \ \ \ \\
\\
\\
c^{T}_{} \ =\ [ 1,\ -1\ ,1]\\
\\
A\ =\ \begin{pmatrix}
2 & -1 & 2\\
2 & -3 & 1\\
-1 & 1 & -2
\end{pmatrix}\\
\\
b\ =\ [ 4,\ -5,\ -1\ ]\\
\\
l\ =\ [ 1,\ 1,\ 2]
\end{array}

In [18]:
model = ConcreteModel()


# Creating number of rows and columns for indexing 

N = 3
M = 3

col_index = np.arange(N)
row_index = np.arange(M)

# Creating model and constrainsts

model.x = Var(col_index)
model.cons = ConstraintList()       # -> Initialising the constraint list

# Creatinf coefficients for objective funtion
coeff_obj_c = np.array([1, -1, 1])


# Coefficient matrix 'A' for constraints, rhs vector 'b' , and lower bound vector 'l' for x 
coeff_matrix_A = np.array([
                           [2, -1, 2],
                           [2, -3, 1],
                           [-1, 1, -2]
                           ])

rhs_cons_b = np.array([4, -5, -1])
lower_bound_x = np.array([1, 1, 2])


# Adding model objective and constraints 
model.objective = Objective(expr =sum(coeff_obj_c[i] * model.x[i] for i in col_index), sense=minimize )

for i in row_index:
  model.cons.add(sum(coeff_matrix_A[i,j] * model.x[j] for j in col_index ) <= rhs_cons_b[i])


for i in col_index:
  model.x[i].setlb(lower_bound_x[i])
  

model.pprint()

2 Set Declarations
    cons_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :  None : False :  True :  Reals
          1 :     1 :  None :  None : False :  True :  Reals
          2 :     2 :  None :  None : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] - x[1] + x[2]

1 Constraint Declarations
    cons : Size=3, Index=cons_index, Active=True
        Key : Lower : Body                   : Upper : Active
          1 :  -Inf : 2*x[0] - x[1] + 2*x[2] :   4.0 :   True
          2 :  -In

In [19]:
# Solving the model with the solver


glpk_solver = SolverFactory('glpk', executable='/usr/bin/glpsol')
cbc_solver =  SolverFactory('cbc')

glpk_result = glpk_solver.solve(model)
cbc_result = cbc_solver.solve(model)

print(glpk_result)
print('\n\n\n\n',cbc_result)



    model=unknown;
        message from solver=<undefined>

Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 10
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: other
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.01387929916381836





 
Problem: 
- Name: unknown
  Lower bound: None
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 4
  Number of nonzeros: 3
  Sense: minimize
Solver: 
- Status: warning
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: unbounded
  Termination message: Model was proven to be unbounded.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc:

# Ex 1 Part 1 

Check the solver status and solver termination criterion for both solvers. Are they meaningful? Explain possible reasons for the solver status and solver ermination criterion message.

**Resons given below**

In [20]:
print('Solver status for GLPK:', glpk_result.solver.status)
print('Solver termination condition for GLPK:',glpk_result.solver.termination_condition)


print('\n\n')

print('Solver status for CBC:', cbc_result.solver.status)
print('Solver termination condition for CBC:',cbc_result.solver.termination_condition)
print('Solver termination message for CBC: ',cbc_result.solver.termination_message)

Solver status for GLPK: ok
Solver termination condition for GLPK: other



Solver status for CBC: warning
Solver termination condition for CBC: unbounded
Solver termination message for CBC:  Model was proven to be unbounded.


**Remarks**

The solver is not able to find the solution because the region bounded by the constraints is not actually bounded(not bounded below). so solution is not feasible in this problem since we are minimizing. That's why the solver status says warning(in cbc) and other(in glpk). And hence the message.

# Ex 1 Part 2 (a)

In this part we will take the following steps:



*   Change the problem to maximizing problem
*   Solve the solver and print the results.
*   Change the problem again to original form i.e back to form in part 1.




In [21]:
#changing the problem to a maximizing problem
model.objective.set_sense(maximize)


#print the model to verify the change
model.pprint()
print('\n')
print('\n')
print('\n')

#solving the solver again to see change
cbc_solver =  SolverFactory('cbc')
cbc_result = cbc_solver.solve(model)

#Results

print('Solver status for CBC:', cbc_result.solver.status)
print('Solver termination condition for CBC:',cbc_result.solver.termination_condition)
print('Solver termination message for CBC: ',cbc_result.solver.termination_message)



print('\n')
print('\n')
print('\n')


print('The optimal value of the objective function is : ',model.objective())
print('\n')
print('The optimal value of the variables are : ' )
print('\n')
for i in col_index:
  print('x[',i,'] : ',model.x[i].value)

model.cons.display()


#Changing the model to original form i.e to part 1 form
model.objective.set_sense(minimize)


2 Set Declarations
    cons_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :  None : False :  True :  Reals
          1 :     1 :  None :  None : False :  True :  Reals
          2 :     2 :  None :  None : False :  True :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : x[0] - x[1] + x[2]

1 Constraint Declarations
    cons : Size=3, Index=cons_index, Active=True
        Key : Lower : Body                   : Upper : Active
          1 :  -Inf : 2*x[0] - x[1] + 2*x[2] :   4.0 :   True
          2 :  -In

In this case the the bounded region was feasible (bounded above) so solver could find the optimal solution easily. The reason for this is that we have changed the objective to maximizing, so the problem has been solved beacause region is bounded above and we can find the maximum value.



> **Constraints that are active with their value :**


*  **Constraint 1 with value 4**

*  **Constraint 2 with value -5** 

# Ex.1 Part 2 (b)

In this part we will take the following steps:


*   Create a model instance named as **'model_2b'**.
*   Add an upper bound on x2 as x2 ≤ 5 in **model_2b**.
*   Solve the solver and print the results for **model_2b**.
*   **Our original model in part 1 remains intact**

In [22]:
#Creating a model instance as model_2b
model_2b = model.create_instance()

#Adding the upper bound to variable x2 in the model_2b
model_2b.x[1].setub(5)

#print the model to verify the change
model_2b.pprint()
print('\n')
print('\n')
print('\n')

#solving the solver again to see change
cbc_solver =  SolverFactory('cbc')
cbc_result_2b = cbc_solver.solve(model_2b)

#Results

print('Solver status for CBC:', cbc_result_2b.solver.status)
print('Solver termination condition for CBC:',cbc_result_2b.solver.termination_condition)
print('Solver termination message for CBC: ',cbc_result_2b.solver.termination_message)



print('\n')
print('\n')
print('\n')


print('The optimal value of the objective function is : ',model_2b.objective())
print('\n')
print('The optimal value of the variables are : ' )
print('\n')
for i in col_index:
  print('x[',i,'] : ',model_2b.x[i].value)


print('\n')
model_2b.cons.display()




    model; returning a clone of the current model instance. (called from
    <ipython-input-22-1691620f37e6>:2)
2 Set Declarations
    cons_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {1, 2, 3}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :   1.0 :  None : False : False :  Reals
          1 :     1 :   3.2 :     5 : False : False :  Reals
          2 :     2 :   2.6 :  None : False : False :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] - x[1] + x[2]

1 Constraint Declarations
    cons : Size=3, Index=cons_index, Active=True
        Key : Lower : Body    


The solver was able to find the optimal solution. Bounded regions.

> **ACTIVE Constraints with their value**


* ***Constraint 3 with value -1***

# Ex.1 Part 2 (c)

In this part we will take the following steps:


*   Create a model instance named as **'model_2c'**.
*   Add a new constraint x2 − x3 ≤ 6 and re-solve **model_2c**.
*   Solve the solver and print the results for **model_2c**.
*   **Our original model in part 1 remains intact**

In [23]:
#Creating a model instance as model_2b
model_2c = model.create_instance()

#Adding the new constraint
model_2c.cons.add(model_2c.x[1] - model_2c.x[2] <= 6)

#print the model to verify the change
model_2c.pprint()
print('\n')
print('\n')
print('\n')

#solving the solver again to see change
cbc_solver =  SolverFactory('cbc')
cbc_result_2c = cbc_solver.solve(model_2c)

#Results

print('Solver status for CBC:', cbc_result_2c.solver.status)
print('Solver termination condition for CBC:',cbc_result_2c.solver.termination_condition)
print('Solver termination message for CBC: ',cbc_result_2c.solver.termination_message)



print('\n')
print('\n')
print('\n')


print('The optimal value of the objective function is : ',model_2c.objective())
print('\n')
print('The optimal value of the variables are : ' )
print('\n')
for i in col_index:
  print('x[',i,'] : ',model_2c.x[i].value)


print('\n')
model_2c.cons.display()




    model; returning a clone of the current model instance. (called from
    <ipython-input-23-ddacd3b76e7b>:2)
2 Set Declarations
    cons_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :   1.0 :  None : False : False :  Reals
          1 :     1 :   3.2 :  None : False : False :  Reals
          2 :     2 :   2.6 :  None : False : False :  Reals

1 Objective Declarations
    objective : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] - x[1] + x[2]

1 Constraint Declarations
    cons : Size=4, Index=cons_index, Active=True
        Key : Lower : Body 

Bounded regions and optimal solution found.

> ***ACTIVE Constraints with their value:***

* **Constraint 3 with value -1**

* **Constraint 4 with value 6**

# Ex.1 Part 2 (d)

In this part we will take the following steps:


*   Create a model instance named as **'model_2d'**.
*   Deactivate objective and add new objective of the form min x1 + x2 +x3 in **model_2d**.
*   Deactivate the constraint  2x1 − 3x2 + x3 ≤ −5 in **model_2d**
*   Add new constraint x1 + x2 ≥ 25 in **model_d**
*   Solve the solver and print the results for **model_2d**.
*   **Our original model in part 1 remains intact**

In [24]:
#Creating a model instance as model_2b
model_2d = model.create_instance()

#Deactivate objective and adding a new
model_2d.objective.deactivate()
model_2d.obj_new = Objective(expr = model_2d.x[0] + model_2d.x[1] + model_2d.x[2], sense=minimize)


#Deactivating Adding the new constraint
model_2d.cons[2].deactivate()
model_2d.cons.add(model_2d.x[0] + model_2d.x[1] >= 25)


#print the model to verify the change
model_2d.pprint()
print('\n')
print('\n')
print('\n')

#solving the solver again to see change
cbc_solver =  SolverFactory('cbc')
cbc_result_2d = cbc_solver.solve(model_2d)

#Results

print('Solver status for CBC:', cbc_result_2d.solver.status)
print('Solver termination condition for CBC:',cbc_result_2d.solver.termination_condition)
print('Solver termination message for CBC: ',cbc_result_2d.solver.termination_message)



print('\n')
print('\n')
print('\n')


print('The optimal value of the objective function is : ',model_2d.obj_new())
print('\n')
print('The optimal value of the variables are : ' )
print('\n')
for i in col_index:
  print('x[',i,'] : ',model_2d.x[i].value)


print('\n')
model_2d.cons.display()




    model; returning a clone of the current model instance. (called from
    <ipython-input-24-2daea32ed4df>:2)
2 Set Declarations
    cons_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}

1 Var Declarations
    x : Size=3, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :   1.0 :  None : False : False :  Reals
          1 :     1 :   3.2 :  None : False : False :  Reals
          2 :     2 :   2.6 :  None : False : False :  Reals

2 Objective Declarations
    obj_new : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[0] + x[1] + x[2]
    objective : Size=1, Index=None, Active=False
        Key  : Active : Sense    : Expression
        No


> ***ACTIVE Constraints with their value:***

* **Constraint 1 with value 4**

* **Constraint 3 with value -1**

* **Constraint 4 with value 25**